In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'


In [ ]:
# connect kafka with spark with simple word count example
# run on a terminal after starting kafka
#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic words
#     kafka-console-producer --bootstrap-server localhost:9092 --topic words  

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic word-counts
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic word-counts --from-beginning  

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()

In [4]:
# read from kafka, here spark is consumer for kafka topic called words
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1

kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "words")\
  .option("group.id", "wordcount-group")\
  .load()

In [5]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, ignore other column
linesDf = kafkaDf.selectExpr("CAST(value AS STRING)")
linesDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)



In [7]:
import pyspark.sql.functions as F
# split line into word list
# flatten word list into individual element as output, similar to flatMap

wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")).alias("word") )

wordCountsDf = wordsDf.groupBy("word").count()
wordCountsDf.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [8]:
echoOnconsole = wordCountsDf\
                .writeStream\
                .outputMode("complete")\
                .option("checkpointLocation", "file:///tmp/spark")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [10]:
# now publish the word count result (word, count columns) to kafka topic "word-counts", publish as json format
# {"word": "kafka", "count": 8}

# F is alias for all functions, we can access col by F.col 
import pyspark.sql.functions as F

# convert all the columns into json
# * represent all columns ie word, count, 
# struct create a structure around word, count columns
# to json convert structure to column
# value is Kafka value part of message
wordCountsToKafkaDf = wordCountsDf\
                    .selectExpr("to_json(struct(*)) as value")

wordCountsToKafkaDf.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
# checkpointLocation is for storing local state, for system restart, system failure in between
# ensure to run kafka console consumer for topic word-count, commands are present in top of file
wordCountsToKafkaDf.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "word-counts")\
                    .option("checkpointLocation", "file:///tmp/spark")\
                    .start()
                    